<a href="https://colab.research.google.com/github/NovaConstantine/10KMining/blob/master/predecting_yield_curve_Algeria_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#pip install nelson-siegel-svensson

In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [7]:
os.listdir()

['.config', 'YIELD TO MATURITY DATA MORROCO.xlsx', 'sample_data']

In [8]:
import os
os.getcwd()

'/content'

In [ ]:
algeria_data = pd.read_excel("YIELD TO MATURITY DATA ALGERIA.xlsx")

In [ ]:
morro_data = pd.read_excel("YIELD TO MATURITY DATA ALGERIA.xlsx")

In [ ]:
algeria_data=algeria_data.iloc[1:,:]

In [ ]:
algeria_data

,Date / maturity,13 WEEKS,26 WEEKS,1 YEAR,2 YEARS,3 YEARS,5 YEARS,7 YEARS,10 YEARS,15 YEARS
1,1995-12-31,22.262,_,_,_,_,_,_,_,_
2,1996-01-31,22.181,_,_,_,_,_,_,_,_
3,1996-02-29,22.275,_,_,_,_,_,_,_,_
4,1996-03-31,21.484,21.554,_,_,_,_,_,_,_
5,1996-04-30,21.554,21.448,_,_,_,_,_,_,_
...,...,...,...,...,...,...,...,...,...,...
313,2021-12-31,2.18,3,4.80012,5.10997,5.21075,5.30597,6.43842,5.1435,5.00508
314,2022-01-31,2.18,3,4.80011,5.10998,4.92043,5.30613,6.43842,5.1435,5.00719
315,2022-02-28,2.18,2.999,4.79479,5.10994,4.93009,5.30627,6.43842,5.1435,5.00719
316,2022-03-31,2.18,2.997,4.78734,5.10995,4.90346,5.30649,6.43842,5.1435,5.00719


In [ ]:
algeria_data.set_index('Date/MaturitY', inplace=True)


# Remove percentage signs and convert to numeric
algeria_data = algeria_data.applymap(lambda x: float(x.replace('%', '')) if x != '-' else None)

KeyError: "None of ['Date/MaturitY'] are in the columns"

In [ ]:
algeria_data.set_index('Date/Maturités', inplace=True)

KeyError: "None of ['Date/Maturités'] are in the columns"

In [ ]:
# Convert date columns to datetime
dates  = pd.to_datetime(algeria_data.index, format='%B - %Y')


ValueError: time data "1" doesn't match format "%B - %Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
dates

NameError: name 'dates' is not defined

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
for column in algeria_data.columns:
    plt.plot(dates, algeria_data[column], label=column)


plt.xlabel('Maturity (years)')
plt.ylabel('Yield (%)')
plt.title('Yield Curve Data')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'dates' is not defined

<Figure size 1000x600 with 0 Axes>

Fill missing **values**

In [ ]:
# Fill missing values with the next value
algeria_data = algeria_data.ffill().bfill()

# Display the processed data
print("\nProcessed Data (Missing values filled):")
print(algeria_data)


Processed Data (Missing values filled):
    Date / maturity   13 WEEKS  26 WEEKS   1 YEAR   2 YEARS   3 YEARS  \
1        1995-12-31     22.262         _        _         _         _   
2        1996-01-31     22.181         _        _         _         _   
3        1996-02-29     22.275         _        _         _         _   
4        1996-03-31     21.484    21.554        _         _         _   
5        1996-04-30     21.554    21.448        _         _         _   
..              ...        ...       ...      ...       ...       ...   
313      2021-12-31       2.18         3  4.80012   5.10997   5.21075   
314      2022-01-31       2.18         3  4.80011   5.10998   4.92043   
315      2022-02-28       2.18     2.999  4.79479   5.10994   4.93009   
316      2022-03-31       2.18     2.997  4.78734   5.10995   4.90346   
317      2022-04-30       2.18     2.997  4.78108   5.10995   4.91166   

      5 YEARS   7 YEARS    10 YEARS   15 YEARS  
1           _          _         

In [ ]:

# Plotting
plt.figure(figsize=(10, 6))
for column in algeria_data.columns:
    plt.plot(dates, algeria_data[column], label=column)


plt.xlabel('Maturity (years)')
plt.ylabel('Yield (%)')
plt.title('Yield Curve Data')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'dates' is not defined

<Figure size 1000x600 with 0 Axes>

**LSTM**

In [ ]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(moroco_data)

# Function to create dataset with lookback
def create_dataset(data, look_back=1):
    X, y = [], []
    for i in range(len(data)-look_back-1):
        X.append(data[i:(i+look_back)])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

# Define forecast horizons
horizons = [1, 3, 6, 12]
#look_back = 20  # Number of previous time steps to use as input

# Split data into training and test sets
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

results = {}
for horizon in horizons:
    look_back = horizon
    X_train, y_train = create_dataset(train_data, look_back)
    X_test, y_test = create_dataset(test_data, look_back)

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

    # Build and Train the LSTM Model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(look_back, X_train.shape[2])))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=25))
    model.add(Dense(units=X_train.shape[2]))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=1, epochs=50)

    # Make Predictions
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions.reshape(-1, predictions.shape[-1]))
    y_test_actual = scaler.inverse_transform(y_test.reshape(-1, y_test.shape[-1]))

    # Calculate Evaluation Metrics
    rmse = np.sqrt(mean_squared_error(y_test_actual, predictions))
    mae = mean_absolute_error(y_test_actual, predictions)
    mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100

    results[horizon] = {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}


NameError: name 'moroco_data' is not defined

In [ ]:
# Display the results
for horizon in results:
    print(f'Horizon: {horizon} months')
    print(f"RMSE: {results[horizon]['RMSE']}")
    print(f"MAE: {results[horizon]['MAE']}")
    print(f"MAPE: {results[horizon]['MAPE']}\n")

NameError: name 'results' is not defined

**RNN**

# Nouvelle section

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout

In [ ]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(algeria_data)

# Function to create dataset with lookback
def create_dataset(data, look_back=1):
    X, y = [], []
    for i in range(len(data)-look_back-1):
        X.append(data[i:(i+look_back)])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

# Define forecast horizons
horizons = [1, 3, 6, 12]
#look_back = 20  # Number of previous time steps to use as input

# Split data into training and test sets
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

results = {}
for horizon in horizons:
    look_back = horizon
    print(f"lookback = {look_back}")
    X_train, y_train = create_dataset(train_data, look_back)
    X_test, y_test = create_dataset(test_data, look_back)

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

    # Build and Train the RNN Model
    model = Sequential()
    model.add(SimpleRNN(units=50, return_sequences=True, input_shape=(look_back, X_train.shape[2])))
    model.add(SimpleRNN(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=25))
    model.add(Dense(units=X_train.shape[2]))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=1, epochs=50)

    # Make Predictions
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions.reshape(-1, predictions.shape[-1]))
    y_test_actual = scaler.inverse_transform(y_test.reshape(-1, y_test.shape[-1]))

    # Calculate Evaluation Metrics
    rmse = np.sqrt(mean_squared_error(y_test_actual, predictions))
    mae = mean_absolute_error(y_test_actual, predictions)
    mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100

    results[horizon] = {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

# Display the results
for horizon in results:
    print(f'Horizon: {horizon} months')
    print(f"RMSE: {results[horizon]['RMSE']}")
    print(f"MAE: {results[horizon]['MAE']}")
    print(f"MAPE: {results[horizon]['MAPE']}\n")


TypeError: float() argument must be a string or a real number, not 'Timestamp'

In [ ]:
# Display the results
for horizon in results:
    print(f'Horizon: {horizon} months')
    print(f"RMSE: {results[horizon]['RMSE']}")
    print(f"MAE: {results[horizon]['MAE']}")
    print(f"MAPE: {results[horizon]['MAPE']}\n")

NameError: name 'results' is not defined

**Test for NS + LSTM**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class NelsonSiegelLayer(Layer):
    def __init__(self, tau, **kwargs):
        super(NelsonSiegelLayer, self).__init__(**kwargs)
        self.tau = tau

    def build(self, input_shape):
        # No trainable weights in this layer
        super(NelsonSiegelLayer, self).build(input_shape)

    def call(self, inputs):
        # Nelson-Siegel equation components
        level = inputs[:, 0:1]
        slope = inputs[:, 1:2]
        curvature = inputs[:, 2:3]

        t = tf.convert_to_tensor(self.tau, dtype=tf.float32)
        term1 = 1.0
        term2 = (1.0 - tf.exp(-t / self.tau)) / (t / self.tau)
        term3 = term2 - tf.exp(-t / self.tau)

        term2 = tf.reshape(term2, (1, -1))
        term3 = tf.reshape(term3, (1, -1))

        return level + slope * term2 + curvature * term3

    def compute_output_shape(self, input_shape):
        return input_shape[0], len(self.tau)




**NS model **

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define maturities (tau values)
maturities = np.array([3, 6, 12, 24, 60, 120, 180, 240, 360])

# Build the hybrid LSTM model with Nelson-Siegel layer
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(look_back, X_train.shape[2])))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=3))  # Level, Slope, Curvature

# Add the custom Nelson-Siegel layer
model.add(NelsonSiegelLayer(tau=maturities))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, batch_size=1, epochs=50)


# Make predictions
predictions = model.predict(X_train)

# Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

# Inverse transform the actual test data
y_test_actual = scaler.inverse_transform(y_test)

# Calculate evaluation metrics
rmse = np.sqrt(mean_squared_error(y_test_actual, predictions))
mae = mean_absolute_error(y_test_actual, predictions)
mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}")



NameError: name 'look_back' is not defined

In [ ]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(moroco_data)

# Function to create dataset with lookback
def create_dataset(data, look_back=1):
    X, y = [], []
    for i in range(len(data)-look_back-1):
        X.append(data[i:(i+look_back)])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

# Define forecast horizons
horizons = [1, 3, 6, 12]
#look_back = 20  # Number of previous time steps to use as input

# Split data into training and test sets
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

results = {}
for horizon in horizons:
    look_back = horizon


    X_train, y_train = create_dataset(train_data, look_back)
    X_test, y_test = create_dataset(test_data, look_back)

    # Reshape input to be [samples, time steps, features]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))




    # Define maturities (tau values)
    maturities = np.array([3, 6, 12, 24, 60, 120, 180, 240, 360])

    # Build the hybrid LSTM model with Nelson-Siegel layer
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(look_back, X_train.shape[2])))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=3))  # Level, Slope, Curvature

    # Add the custom Nelson-Siegel layer
    model.add(NelsonSiegelLayer(tau=maturities))

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, batch_size=1, epochs=50)


    # Make Predictions
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions.reshape(-1, predictions.shape[-1]))
    y_test_actual = scaler.inverse_transform(y_test.reshape(-1, y_test.shape[-1]))

    # Calculate Evaluation Metrics
    rmse = np.sqrt(mean_squared_error(y_test_actual, predictions))
    mae = mean_absolute_error(y_test_actual, predictions)
    mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100

    results[horizon] = {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}


NameError: name 'moroco_data' is not defined

In [ ]:

# Display the results
for horizon in results:
    print(f'Horizon: {horizon} months')
    print(f"RMSE: {results[horizon]['RMSE']}")
    print(f"MAE: {results[horizon]['MAE']}")
    print(f"MAPE: {results[horizon]['MAPE']}\n")


NameError: name 'results' is not defined

In [ ]:
pip install pandas numpy scikit-learn keras tensorflow

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

# Charger les données
data = pd.read_csv('obligatory_market_data.csv')

# Préparer les données
# Supposons que les colonnes de data incluent 'Date', 'Maturité' et 'Taux'

# Fonction pour créer les données de séquences pour les RNN/LSTM
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Préparer les données pour le modèle Nelson-Siegel dynamique
# Par exemple, utiliser des caractéristiques comme maturité et taux
features = data[['Maturité', 'Taux']].values
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Définir la longueur de la séquence
seq_length = 10
X, y = create_sequences(features_scaled, seq_length)

# Diviser les données en ensembles d'entraînement et de test
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Créer le modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length, 2)))
model.add(LSTM(units=50))
model.add(Dense(units=2))

# Compiler le modèle
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Entraîner le modèle
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Évaluer le modèle
loss = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')

# Faire des prédictions
y_pred = model.predict(X_test)

# Inverser la transformation pour obtenir les valeurs originales
y_test_original = scaler.inverse_transform(y_test)
y_pred_original = scaler.inverse_transform(y_pred)

# Afficher les résultats
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(y_test_original[:, 1], label='True Rates')
plt.plot(y_pred_original[:, 1], label='Predicted Rates')
plt.title('Actual vs Predicted Interest Rates')
plt.xlabel('Time')
plt.ylabel('Interest Rate')
plt.legend()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'obligatory_market_data.csv'